# Extended USArray Processing Workflow
This notebook has no python code.  It was created to document how the extended usarray data set was created and how to run the notebooks in this directory to process that data.   

## Creation of data set
The data set involved was actually assembled several years ago.   It used a sequence of python scripts to drive obspy's bulk_download function to assemble te waveforms.   An example is found in the subdirectory below the current one called "download_scripts".  Specifics can be found there but there are several points to note here:
1.  The data were assembled in chunks of a years.
2.  The data driving the assembly is a list of teleseismic earthquakes.   That list is obtained by running the python function `download_events` which uses a point defined by the file "year_centers.txt" and finds all events with magnitude > 4.9 at distances from 25 to 95 degrees from that point.
3.  The waveform data and station data were downloaded using obspy driven by the function called `download_events` also found in "rfteledownload.py".  Note inside that function is hard coded the station search window with an ObsPy `RectangularDomain` class.  See the function for the region.

That procedure was marginally feasible with web services a few years ago.  It took approximately 6 months with various starts and stops to assemble these data.  A better metric is with no problems each year took around a week to assemble.   The additional delays were from me figuring out how to deal with the disconnect of what I got from what I needed.  I'm not sure I preserved all the details, but here are some things I know I had to deal with:
1.  The original waveforms were downloaded as millions of single channel files.  That would not have worked at all on an HPC file system, but I did this assembly on an old Mac with a conventional unix file system.   I concatenated files in the event directories the download produced into event files with names like "event1.mseed", "event2.mseed", ...   The miniseed event files were then saved on an archival storage system at Indiana University.
2.  The download created images of the xml data downloaded.   The event data was in single files and the station data was placed into another monster set of files in year directories.  I archived that data, but later learned it was best to simply ignore it and refresh the same data with web services.  See below

I must note that I did my best but I have no way to know if these data are complete.   Several years required multiple runs to get all the data the long running script died for one reason or another.   

## Step 1:   Create master Metadata database
Earlier attempts to deal with creating a working database for site, channel, and source collections went through through two variations I abandoned in favor of what is done here:
1.  I tried using the xml files obspy had downloaded when I first started this years ago.  That was a disaster because of duplicates and the absurd number of files.
2.  Until recently I used a procedure similar to used in various tutorials from MsPASS short courses.   That is, I would download event and station metadata matching the time span of the section of data being processed.

I realized the second approach above was problematic when it came to assembling the full data set from 2005-2015.  I could have merged the pieces, but when I realized I had a similar problem at the end of the sequence of merging all the receiver functions into a common database I realized it made more sense to make a master database for the full project and use the yearlies as more of a scratch database that would not necessarily even be retained.   

With that background step one for processing the entire data set is to run the notebook called "CreateMasterDatabase.ipynb".  I ran that notebook on the cluster as an interactive job using the jupyter lab interface.  The master here then contains the output for running that notebook.  I should have recorded the time elapsed but it was at least an hour.  Running that notebook creates three collections spanning Jan 1, 2005 to Dec 31, 2015:  source, site, and channel.   The source collection is much larger than what I downloaded as the lower magnitude limit was 4.5 versus 4.9 in the download.  I did that to make sure we had all the event data.  It makes the source collection about 4 times bigger than the original.  Still tiny, however, compared to the total number of waveforms of which there are around 50 million

## Step 3:  Preprocessing
### Overview
Preprocessing is defined here as taking the raw miniseed data downloaded earlier and producing a set of Seismogram objects that can be used as input for P to S impulse response estimates.   This workflow forks at that point depending upon how one wants to produce the impulse response estimates.   This workflow preserved here on github finalizes the processing using the new "CNRDecon" function to produce a form of "receiver functions" where the deconvolution is preformed on each Seismogram object without any information from any other data.  I am developing an array deconvolution workflow that may eventually appear in the mspass_tutorials repository but that was not finished when this notebook was written.  

The top structure of this processing is to realize it was designed to be run in one year chunks.  There were two reasons for that choice.  First, it is a natural, simple organization.  Second, the size of the raw data from one year of the extended usarray set of stations is about right to chew on for current generation hardware - around 0.5 to 1 TB per year depending on the years (2011 is about twice as big as other years because of huge number of aftershocks of the Tohoku earthquake in Japan in early 2011.)   For that reason the rest of this main section describes what needs to be run for each year block.

### Yearly Step 1:  retrieve miniseed data from archive
A copy of the miniseed files that define this data set are archived at Indiana University and TACC.  I believe the two copies are identical.   2005 and 2006 will require some special attention and are not discussed in this document I'm posting to github.  They were my original prototypes and I handled them differently.   It will only confuse this tutorial to discuss that different handling.  For now, step 1 is to retrieve the event files from the mass store at TACC or the IU system.  The next phase *requires* the miniseed files to be placed in "year" directories rooted with "./wf" where "." is your selected work directory.  e.g. if you are working on the data from 2012 the miniseed files should be placed in the directory "./wf/2012".  Similarly, 2013 data should be placed in "./wf/2013".   The next step would need to be modified if you change that file organization.  Note the archive file names are fixed and have names like "event1.msdeed", "event2.mseed", ..., "eventN.mseed" where N is the number of events for that year.  


### Year step 2:  Build wf_miniseed
The first serious processing is done by making a copy of the notebook called "index_mseed_template.ipynb" that will become your run file.   I recommend you just change "template" to the version for the year you are processing.  e.g. if you are working on 2012 I would run this in the shell:
```
cp index_mseed_template.ipynb index_mseed_2012.ipynb
```
Then edit the copy you created and edit one line in the first code box of that notebook.  The template currently has this:
```
# change for different calendar year
year = 2014
```
As the comment suggest change the year appropriate - 2012 for the example immediately agove. 

Run that script on your cluster.  This job does not benefit much from a large number of cores and is probably best run on a single node.  The reason is over half the the processor time is spent running the `bulk_normalize` function to create the id cross references for the source, site, and channel collections.   That is not parallelized in this notebook and takes quite some time to run.  Allow 8 hours in a batch job to be sure, but most years will run in less than 2 hours.  

### Year step 3:  TimeSeries processing
The first notebook that does anything more than bookkeeping is the one you run next.  The master is called "Preprocess2ts_template.ipynb".  Like before for the year being processed you should:
1.  Copy the master file changing "template" to the relevant year.
2.  Edit the file changing the "year = 2014" to the relevant years.

If the previous step completed successfully you should be able to run this notebook on the cluster.   A few points about this part of the processing work:
1.  With this set of waveform files as structured here this is large memory job.  The reason is that if you look at the notebook you see that the processing is a loop over common source gathers and each gather is processed in a paralle construct.   The processed gather is merged in a serial loop after collecting all the data from the cluster (what happens when the bag compute method is called).  An advantage to that approach is it speeds output with the binary ensemble writer, but is at the cost of being a large memory algorithm.  At IU I had to configure the job so that each worker had 25 Gb of memory.  On the IU system that meant I had to request 25 Gb per mpi process, set the cpus-per-task to fit in the system's physical memory, and set the number of workers to match.  On IU's bigred200 I set slurm up as follows:
```
#SBATCH --mem-per-cpu=25G
#SBATCH --cpus-per-task=8
```
and launched workers with this incantation:
```
export MSPASS_WORKER_ARG="--nworkers 8 --nthreads 1 --memory-limit=25G"
```
2. Note the notebook writes event files in a new directory called "./wf_TimeSeries" with a year directory parallel to that in wf.  e.g. if you are working on 2012 data the input will be miniseed files in wf/2012 and the processed output will appear in the directory "wf_TimeSeries/2012".   Those files, however, are plain binary fwrite output of all the ensemble sample data.  They are like old fortran unformatted write and are useless without database documents created in the wf_TimeSeries collection.  The documents in wf_TimeSeries are the Metadata describing the sample data stored in the binary files.  The names are the same as the parent miniseed files but we strip the ".mseed" so the files have shorter names like "event1", "event2", etc.
3. This notebook takes the longest time to run of the series.   On IU's bigred200 I requested 12 hours for a run, which was close for most years.  I would double that for 2012 as that year had almost twice as many events due to the Tohoku earthquake as noted above.

### Year step 4:  Create edited Seismogram data
The template for this step is the notebook called "Preprocess2seis_template.ipynb".  As with the previous year templates the top level steps is to copy the file to one for the year, change the year in that file, and run it on the cluster.   

This workflow is the most compute bound job of the series and will likely scale best if run on many nodes.   The main bottleneck is running the function called `broadband_snr_QC`.   That function does a lot of computation to compute a long list of metrics measuring the quality of a signal using a set of different measures of signal-to-noise ratio.  The job is not at all IO bound from what I can tell because it reads and write ensembles in the binary format, which is the fastest IO method currently available with MsPASS and the computational load of `broadband_snr_QC` is not trivial. 

This workflow creates a "wf_Seismgram" collection in the year database much like the previous step creates a "wf_TimeSeries" collection for that year  It also writes binary files similar to those in wf_TimeSeries but containing the Seismgoram objects grouped in source files.  They are direct descendents but not necessarily in the same order.  e.g. the sample data in file "./wf_TimeSeries/2012/event4" becomes the data stored in "./wf_Seismogram/2012/event4" but rearranged to match Seismogram objects and modified by this workflow.   

This notebook scales well with multiple processors.  For instance, the 2013 data took less than 2 hours to run with 3 nodes at IU.  

### Year Step 5:   Receiver Function Estimation
The final step is to run a python script called "CNRRFDecon.py".  As with other components you need to change the `year = xxxx` to the appropriate value.  A difference at the momement is this part of the workflow is a pure python script and not in a notebook.  Otherwise, you should recognize the same block of code at the top of the python script.  

A couple relevant details about running this script:
1.  The file "CNRDeconEngine.pf" is required to run this script.  It is an "pf-file" that defines a fairly extensive set of parameters that are required for the algorithm to run.
2.  This script is serial and is a waste if run on more than a single node.  The reason is we have a serialization issue with `CNRDeconEngine` that we have been unable to resolve.  Yearly results, in my experience, take of the order of one hour to run so the serial algorithm is feasible.  If submitted batch I would suggest requesting a 2 hour block to assure completion.   